## Combining All Monthly/Daily Data to SIngle DataFrame

    - also checks for mismatch or inconsistency in data format

In [ ]:
import pandas as pd
from tqdm import tqdm

In [ ]:
def convert_to_date(x):
    s = str(x)
    s = s.strip()
    s = s.split(" ")[0]
    if len(s.split("-")[0]) == 4:
        return pd.to_datetime(x, format='%Y-%d-%m %H:%M:%S')
    elif len(s.split("-")[2]) == 4:
        return pd.to_datetime(x, format='%d-%m-%Y %H:%M:%S')
    else:
        print(s, s.split("-")[2])
        sys.exit()
        
def get_hod(row):
    x = row['Raised Time']
    s = str(x)
    s = s.strip()
    s = s.split(" ")[0]
    if len(s.split("-")[0]) == 4:
        dts = pd.to_datetime(x, format='%Y-%d-%m %H:%M:%S')
        return dts.hour
    elif len(s.split("-")[2]) == 4:
        dts = pd.to_datetime(x, format='%d-%m-%Y %H:%M:%S')
        return dts.hour
    else:
        print(s, s.split("-")[2])
        sys.exit()


In [ ]:
data_dir = 'data/Alarms/'
os.listdir(data_dir)

In [ ]:
df_ = pd.read_excel(
             os.path.join(data_dir, 'Alarms_October2020.xlsx'),
             engine='openpyxl',
             converters= {'Raised Time': convert_to_date, 'Resolved Time': convert_to_date}
        )


In [ ]:
df_

In [ ]:
data_dir = 'data/Alarms/'
# adata = pd.ExcelFile(os.path.join(data_dir, 'Alarms_April2019.xlsx'))
# sheets = adata.sheet_names
# print(sheets)
# sheet_index = sheets.index('Alarms')
# df1 = pd.read_excel(adata, 
#                         sheets[sheet_index], 
#                         sep=r'\s*,\s*', 
#                         converters= {'Raised Time': convert_to_date, 'Resolved Time': convert_to_date})

all_dfs = []
for file in tqdm(os.listdir(data_dir)):
    df_ = pd.read_excel(
                 os.path.join(data_dir, file),
                 engine='openpyxl',
                 converters= {'Raised Time': convert_to_date, 'Resolved Time': convert_to_date}
            )
    try:
        df_ = df_[df_["Condition Name"].str.contains("Inverter event")]
        df_ = df_[~df_['Message'].isnull()] # remove null strings
        df_["Error Code"] = df_["Message"].apply(lambda x: x.split(": ")[1])
        df_['hod'] = df_.apply(get_hod, axis=1)
    #     df__ = df_[(df_.hod >= 6) & (df_.hod <= 17)]
        all_dfs.append(df_)
    except:
        print("Format mismatch", file)

In [ ]:
df_combined = pd.concat(all_dfs, axis=0)


In [ ]:
df_combined

In [ ]:
dfc2 = df_combined[(df_combined.hod >= 6) & (df_combined.hod <= 17)]
dfg = dfc2.groupby('Error Code')
dfg["Controller Name"].count().reset_index().sort_values(['Controller Name'], ascending=False)

In [ ]:
df_combined.to_csv("all_alarms.csv", index=False, header=True)

In [ ]:
df_

In [ ]:
print(df1.shape, df_temp2.shape)

In [ ]:
df1 = pd.read_excel(
             os.path.join(data_dir, 'Alarms_April2019.xlsx'),
             engine='openpyxl',
             converters= {'Raised Time': convert_to_date, 'Resolved Time': convert_to_date}
        )
df1 = df1[df1["Condition Name"].str.contains("Inverter event")]


In [ ]:
df1

In [ ]:
import pandas as pd
import os
from tqdm import tqdm

In [ ]:
data_dir = 'data/SMB/'
# df_ = pd.read_csv(os.path.join(data_dir, 'min180501_smb.csv'))
all_dfs = []
year = '18'
for file in tqdm(os.listdir(data_dir)):
    year_f = file[3:5]
    if year_f != year:
        df_combined = pd.concat(all_dfs, axis=0)
        df_combined.to_csv("all_smbs_"+year+".csv", index=False, header=True)
        all_dfs = []
        year = year_f
    try:
        df_ = pd.read_csv(os.path.join(data_dir, file))
        df_["date"] = pd.to_datetime(df_["Time"])
        all_dfs.append(df_)
    except:
        print("problem", file)
df_combined = pd.concat(all_dfs, axis=0)
df_combined.to_csv("all_smbs_"+year+".csv", index=False, header=True)

In [ ]:
df_.head()

In [ ]:
df_.columns.tolist()

In [ ]:
# data format is different for the month of August'2019
# for all the other months pd.to_datetime() without any format is sufficient
# for Aug'19 we have to use: pd.to_datetime('01-08-2019 00:00', format='%d-%m-%Y %H:%M')
# min1908xx.csv'
# same for the four days of September'19
data_dir = 'data/Inverter'
all_dfs = []
for file in tqdm(os.listdir(data_dir)):
    try:
        df_ = pd.read_csv(os.path.join(data_dir, file))
        if file[3:7] == '1908' or file[3:7] == '1909':
            df_["date"] = pd.to_datetime(df_["Time"], format='%d-%m-%Y %H:%M')
        else:
            df_["date"] = pd.to_datetime(df_["Time"])
        all_dfs.append(df_)
    except:
        print("problem", file)
df_combined = pd.concat(all_dfs, axis=0)
df_combined.to_csv("all_inverters.csv", index=False, header=True)

In [ ]:
df_combined.head()